# DSPy Optimization for ANEETAA Agents

This notebook demonstrates how to optimize ANEETAA agents using DSPy's SIMBA optimizer.

## Workflow:
1. Prepare training data from NEET materials
2. Define DSPy signatures and modules
3. Run SIMBA optimizer
4. Evaluate optimized agents
5. Log to MLflow
6. Deploy optimized models

In [2]:
# Install dependencies if needed
%pip install --upgrade "pydantic>=2.0" "dspy>=3.0.3" "mlflow>=3.4.0" datasets pandas numpy python-dotenv scikit-learn

  Using cached pydantic-2.12.3-py3-none-any.whl.metadata (87 kB)
  Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.3.4-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached pydantic-2.12.3-py3-none-any.whl (462 kB)
Using cached pandas-2.3.3-cp311-cp311-win_amd64.whl (11.3 MB)
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   -------------------------------------- - 8.7/8.9 MB 44.9 MB/s eta 0:00:01
   ---------------------------------------- 8.9/8.9 MB 42.5 MB/s  0:00:00
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)

  Attempting uninstall: threadpoolctl

    Found existing installation: threadpoolctl 2.2.0

    Uninstalling threadpoolctl-2.2.0:

   ---------------------------------------- 0/5 [threadpoolctl]
   ---------------------------------------- 0/5 [threadpoolctl]
   ---------------------------------------- 0/5 [threadpoolctl]
   --

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.3.35 requires langchain-core<1.0.0,>=0.3.78, but you have langchain-core 0.2.43 which is incompatible.


In [1]:
import os
import sys
import json
import dspy
import mlflow
import pandas as pd
import numpy as np
from pathlib import Path

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

print("✓ Imports successful")

c:\Users\benja\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:61: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


✓ Imports successful


## 1. Setup DSPy and MLflow

In [5]:
# Configure DSPy with your LLM
import getpass

# For OpenAI (recommended for optimization)
api_key = getpass.getpass("Enter your OpenAI API key: ")
os.environ['OPENAI_API_KEY'] = api_key

lm = dspy.LM(
    model="openai/gpt-4o-mini",
    max_tokens=500,
    temperature=0.1
)
dspy.settings.configure(lm=lm)

print("✓ DSPy configured with", lm.model)

✓ DSPy configured with openai/gpt-4o-mini


In [3]:
# Setup MLflow (safe: fall back to local file store to avoid network timeouts)
from pathlib import Path
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

# Load .env if present (so teammate's .env is respected)
load_dotenv()

# Use environment variable if set, otherwise use a local `mlruns` folder to avoid hitting an unavailable server
mlflow_tracking_uri = os.getenv('MLFLOW_TRACKING_URI', f"file://{str(Path.cwd() / 'mlruns')}")
mlflow.set_tracking_uri(mlflow_tracking_uri)

# If MLFLOW_EXPERIMENT_ID is set, use it; otherwise use experiment name
exp_id = os.getenv('MLFLOW_EXPERIMENT_ID')
if exp_id:
    try:
        client = MlflowClient()
        exp = client.get_experiment(exp_id)
        if exp is not None:
            mlflow.set_experiment(experiment_id=exp_id)
            print(f"Using Databricks experiment: {exp.name} (ID: {exp_id})")
        else:
            print(f"Experiment ID {exp_id} not found, creating new experiment")
            mlflow.set_experiment('aneeta-dspy-optimization')
    except Exception as e:
        print(f"Error accessing experiment ID: {e}")
        mlflow.set_experiment('aneeta-dspy-optimization')
else:
    mlflow.set_experiment('aneeta-dspy-optimization')

# Attempt to enable DSPy autologging if available, but fail fast and continue if not
try:
    if hasattr(mlflow, 'dspy'):
        try:
            mlflow.dspy.autolog()
            print("✓ DSPy autolog enabled")
        except Exception as e:
            print(f"ℹ DSPy autolog not available: {e}")
    else:
        print("ℹ mlflow.dspy not present in MLflow; check installation")
except Exception as e:
    print(f"⚠ MLflow autolog setup error: {e}")

print("✓ MLflow configured")
print(f"  Tracking URI: {mlflow.get_tracking_uri()}")

Using Databricks experiment: /Users/benjaminloh.2023@smu.edu.sg/MLflow Tracing Tutorial (ID: 3840367859585746)
✓ DSPy autolog enabled
✓ MLflow configured
  Tracking URI: databricks


In [4]:
# Diagnostic cell: verify MLflow <-> Databricks connectivity and DSPy tracer availability
import os
import mlflow
import dspy
from dotenv import load_dotenv
from mlflow.tracking import MlflowClient

load_dotenv()

print('mlflow version:', getattr(mlflow, '__version__', 'unknown'))
print('dspy version:', getattr(dspy, '__version__', 'unknown'))
print('mlflow.dspy present:', hasattr(mlflow, 'dspy'))
print('Effective tracking URI (before set):', mlflow.get_tracking_uri())

# If notebook or environment set a tracking URI, mlflow.get_tracking_uri() will show it.
# Try to resolve the provided experiment id (if any) to a name on the tracking server
exp_id = os.getenv('MLFLOW_EXPERIMENT_ID')
if exp_id:
    try:
        client = MlflowClient()
        exp = client.get_experiment(exp_id)
        if exp is not None:
            print(f'Resolved experiment id={exp_id} -> name={exp.name}')
        else:
            print(f'Experiment id={exp_id} not found on tracking server')
    except Exception as e:
        print('Failed to resolve experiment id:', e)
else:
    print('No MLFLOW_EXPERIMENT_ID set in environment')

# If mlflow.dspy exists, print a quick attribute to ensure it is usable
if hasattr(mlflow, 'dspy'):
    try:
        print('mlflow.dspy OK:', hasattr(mlflow.dspy, 'autolog'))
    except Exception as e:
        print('mlflow.dspy present but accessing it raised:', e)


mlflow version: 2.18.0
dspy version: 3.0.3
mlflow.dspy present: True
Effective tracking URI (before set): databricks
Resolved experiment id=3840367859585746 -> name=/Users/benjaminloh.2023@smu.edu.sg/MLflow Tracing Tutorial
mlflow.dspy OK: True
Resolved experiment id=3840367859585746 -> name=/Users/benjaminloh.2023@smu.edu.sg/MLflow Tracing Tutorial
mlflow.dspy OK: True


## 2. Load Training Data

In [6]:
def load_neet_training_data():
    """Load training data from processed NEET materials."""
    data_dir = Path.cwd().parent / 'Processed Data'
    
    # Load biology chunks
    bio_path = data_dir / 'processed_biology_chunks.json'
    chem_path = data_dir / 'processed_chemistry_chunks.json'
    physics_path = data_dir / 'processed_physics_chunks.json'
    
    examples = []
    
    for path, subject in [(bio_path, 'biology'), (chem_path, 'chemistry'), (physics_path, 'physics')]:
        if path.exists():
            with open(path, 'r', encoding='utf-8') as f:
                chunks = json.load(f)
                
            # Convert chunks to Q&A format (simple heuristic)
            for chunk in chunks[:20]:  # Limit for demo
                content = chunk.get('page_content', '')
                if len(content) > 100:
                    # Create a question from the first sentence
                    sentences = content.split('.')
                    if len(sentences) >= 2:
                        question = f"Explain: {sentences[0].strip()}"
                        answer = content
                        
                        examples.append(dspy.Example(
                            question=question,
                            context=content,
                            answer=answer[:500],
                            subject=subject
                        ).with_inputs('question', 'context'))
    
    print(f"✓ Loaded {len(examples)} training examples")
    return examples

# Load data
training_data = load_neet_training_data()

# Split into train/test
from sklearn.model_selection import train_test_split
trainset, testset = train_test_split(training_data, test_size=0.2, random_state=42)

print(f"Train: {len(trainset)} | Test: {len(testset)}")

✓ Loaded 58 training examples
Train: 46 | Test: 12
Train: 46 | Test: 12


In [7]:
# Inspect a training example
print("Example training data:")
print(trainset[0])

Example training data:
Example({'question': 'Explain: 147 Appendix\nIn each case, we divide or multiply the decimal by a power of 10, to bring one non-zero digit to the left\nof the decimal point, and do the reverse operation by the same power of 10, indicated separately', 'context': '147 Appendix\nIn each case, we divide or multiply the decimal by a power of 10, to bring one non-zero digit to the left\nof the decimal point, and do the reverse operation by the same power of 10, indicated separately.\nThus, any positive decimal can be written in the form\nn = m × 10\np\nwhere p is an integer (positive, zero or negative) and 1 < m < 10. This is called the “standard form of n.”\nWorking Rule\n1. Move the decimal point to the left, or to the right, as may be necessary, to bring one non-zero digit\nto the left of decimal point.\n2. (i) If you move p places to the left, multiply by 10\np\n.\n(ii) If you move p places to the right, multiply by 10\n–p\n.\n(iii) If you do not move the decimal p

## 3. Define DSPy Modules

In [8]:
# Import from our DSPy agents module
from aneeta.nodes.agents_dspy import (
    TeacherSignature,
    TeacherAgentDSPy,
    MCQSolverSignature,
    MCQSolverAgentDSPy
)

# Initialize agents
teacher_agent = TeacherAgentDSPy()
mcq_solver = MCQSolverAgentDSPy()

print("✓ DSPy agents initialized")

2025-10-31 01:45:32.024 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 01:45:32.427 
  command:

    streamlit run c:\Users\benja\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-31 01:45:32.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 01:45:32.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 01:45:32.429 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 01:45:32.427 
  command:

    streamlit run c:\Users\benja\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-10-31 01:45:32.428 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-31 01:45:32.428 Thread 'MainThrea

✓ DSPy agents initialized


## 4. Define Evaluation Metrics

In [9]:
def validate_explanation(example, prediction, trace=None):
    """Validate if explanation is good quality."""
    # Simple check: response should be longer than question
    response = prediction.response if hasattr(prediction, 'response') else str(prediction)
    
    if len(response) < 50:
        return 0.0
    
    # Check if it contains key terms from context
    context_words = set(example.context.lower().split())
    response_words = set(response.lower().split())
    overlap = len(context_words & response_words) / len(context_words) if context_words else 0
    
    return overlap > 0.1

print("✓ Metrics defined")

✓ Metrics defined


## 5. Optimize Teacher Agent with SIMBA

In [10]:
from dspy import SIMBA

# Configure optimizer
optimizer = SIMBA(
    metric=validate_explanation,
    max_demos=3,        # Number of demonstrations to bootstrap
    bsize=12,           # Batch size for evaluation
    num_threads=1       # Number of parallel threads
)

print("✓ SIMBA optimizer configured")
print("Starting optimization... (this may take a few minutes)")

✓ SIMBA optimizer configured
Starting optimization... (this may take a few minutes)


In [11]:
# Run optimization
with mlflow.start_run(run_name="teacher_agent_optimization"):
    optimized_teacher = optimizer.compile(
        teacher_agent,
        trainset=trainset[:30],  # Use subset for faster optimization
    )
    
    print("✓ Optimization complete!")

2025/10/31 01:45:48 INFO dspy.teleprompt.simba: Starting batch 1 of 8.
2025/10/31 01:45:48 INFO dspy.teleprompt.simba: Sampling program trajectories on 12 examples x 6 samples.
2025/10/31 01:45:48 INFO dspy.teleprompt.simba: Sampling program trajectories on 12 examples x 6 samples.


Processed 60 / 72 examples:  82%|████████▏ | 59/72 [00:00<00:00, 235.21it/s]TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
Processed 71 / 72 examples:  97%|█████████▋| 70/72 [00:00<00:00, 235.21it/s]TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
Processed 72 / 72 examples: 100%|██████████| 72/72 [00:00<00:00, 232.76it/s]

2025/10/31 01:45:48 INFO dspy.teleprompt.simba: Batch 1: Baseline mini-batch score: 0.0

2025/10/31 01:45:48 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #1, with max score 0.0, max-to-min gap 0.0, and max-to-avg gap 0.0.
2025/10/31 01:45:48 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_demo_
2025/10/31 01:45:48 INFO dspy.teleprompt.simba_utils: Added 0 demos (one each) across all predictors.
2025/10/31 01:45:48 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #1, with max score 0.0, max-to-min gap 0.0, and max-to-avg gap 0.0.
2025/10/31 01:45:48 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_demo_
2025/10/31 01:45:48 INFO dspy.teleprompt.simba_utils: Added 0 demos (one each) across all predictors.
2025/10/31 01:45:48 INFO dspy.teleprompt.simba: 

2025/10/31 01:45:48 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #2, with max score 0.0, max-to-min gap 0.0, and max-to-avg gap 0.0.
2025/10/31 01:45:48 INFO dspy.teleprompt.simba

2025/10/31 01:45:57 INFO dspy.teleprompt.simba_utils: Advice for generate_explanation.predict: If the module receives a complex question about physics concepts, it should break down the question into simpler components, ensuring that each part is addressed clearly. Additionally, it should leverage the context provided to extract key terms and concepts, ensuring that the explanation is both accurate and comprehensible. Avoid overly technical jargon unless necessary, and always aim for clarity in the response.
2025/10/31 01:45:57 INFO dspy.teleprompt.simba: 

2025/10/31 01:45:57 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #3, with max score 0.0, max-to-min gap 0.0, and max-to-avg gap 0.0.
2025/10/31 01:45:57 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_demo_
2025/10/31 01:45:57 INFO dspy.teleprompt.simba_utils: Added 0 demos (one each) across all predictors.
2025/10/31 01:45:57 INFO dspy.teleprompt.simba: 

2025/10/31 01:45:57 INFO dspy.teleprompt.simba: Ba

Processed 46 / 84 examples:  54%|█████▎    | 45/84 [00:00<00:00, 251.64it/s]TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
Processed 53 / 84 examples:  62%|██████▏   | 52/84 [00:00<00:00, 228.65it/s]TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
Processed 73 / 84 examples:  86%|████████▌ | 72/84 [00:00<00:00, 228.65it/s]TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
Processed 84 / 84 examples: 100%|██████████| 84/84 [00:00<00:00, 239.20it/s]

...

Processed 84 / 84 examples: 100%|██████████| 84/84 [00:00<00:00, 143.33it/s]

2025/10/31 01:47:36 INFO dspy.teleprompt.simba: Scores after 8 batches: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Best: 0.0

2025/10/31 01:47:36 INFO dspy.teleprompt.simba: VALIDATION: Evaluating 7 programs on the full trainset.
2025/10/31 01:47:36 INFO dspy.teleprompt.simba: VALIDATION: Evaluating 7 programs on the full trainset.



Processed 3 / 210 examples:   1%|          | 2/210 [00:00<00:01, 140.27it/s]TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
Processed 78 / 210 examples:  37%|███▋      | 77/210 [00:00<00:00, 189.58it/s]TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
Processed 109 / 210 examples:  51%|█████▏    | 108/210 [00:00<00:00, 194.14it/s]TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
TeacherAgentDSPy.forward() missing 1 required positional argument: 'language'
Processed 210 / 210 examples: 100%|██████████| 210/210 [00:01<00:00, 203.28it/s]

2025/10/31 01:47:38 INFO dspy.teleprompt.simba: Final trainset scores: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Best: 0.0 (at index 0)






✓ Optimization complete!
✓ Optimization complete!
🏃 View run teacher_agent_optimization at: https://dbc-847d0b13-8d35.cloud.databricks.com/ml/experiments/3840367859585746/runs/43de6b0835c54675907bf916a18c219e
🧪 View experiment at: https://dbc-847d0b13-8d35.cloud.databricks.com/ml/experiments/3840367859585746
🏃 View run teacher_agent_optimization at: https://dbc-847d0b13-8d35.cloud.databricks.com/ml/experiments/3840367859585746/runs/43de6b0835c54675907bf916a18c219e
🧪 View experiment at: https://dbc-847d0b13-8d35.cloud.databricks.com/ml/experiments/3840367859585746


## 6. Evaluate Performance

In [12]:
def evaluate_agent(agent, testset, name="Agent"):
    """Evaluate agent on test set."""
    scores = []
    
    for example in testset[:10]:  # Limit for demo
        try:
            prediction = agent(
                question=example.question,
                context=example.context,
                language="English"
            )
            score = validate_explanation(example, prediction)
            scores.append(score)
        except Exception as e:
            print(f"Error: {e}")
            scores.append(0)
    
    avg_score = np.mean(scores)
    print(f"{name} - Average Score: {avg_score:.2%}")
    return avg_score

# Evaluate baseline vs optimized
baseline_score = evaluate_agent(TeacherAgentDSPy(), testset, "Baseline")
optimized_score = evaluate_agent(optimized_teacher, testset, "Optimized")

improvement = ((optimized_score - baseline_score) / baseline_score * 100) if baseline_score > 0 else 0
print(f"\n🎉 Improvement: {improvement:.1f}%")

2025/10/31 01:49:35 WARNING dspy.utils.callback: Error when calling callback <mlflow.dspy.callback.MlflowCallback object at 0x000001F8670BE6D0>: 'ChainOfThought' object has no attribute 'signature'
2025/10/31 01:49:35 WARNING mlflow.dspy.callback: Failed to create a span. Parent span not found.
2025/10/31 01:49:35 WARNING mlflow.dspy.callback: Failed to create a span. Parent span not found.
2025/10/31 01:49:44 WARNING mlflow.dspy.callback: Failed to end a span. Span not found for call_id: 81c8676e885d45acabafa000ce5c93f5
2025/10/31 01:49:44 WARNING mlflow.dspy.callback: Failed to end a span. Span not found for call_id: 81c8676e885d45acabafa000ce5c93f5
2025/10/31 01:49:48 WARNING dspy.utils.callback: Error when calling callback <mlflow.dspy.callback.MlflowCallback object at 0x000001F8670BE6D0>: 'ChainOfThought' object has no attribute 'signature'
2025/10/31 01:49:48 WARNING dspy.utils.callback: Error when calling callback <mlflow.dspy.callback.MlflowCallback object at 0x000001F8670BE6D0

Baseline - Average Score: 90.00%


2025/10/31 01:51:52 WARNING dspy.utils.callback: Error when calling callback <mlflow.dspy.callback.MlflowCallback object at 0x000001F8670BE6D0>: 'ChainOfThought' object has no attribute 'signature'
2025/10/31 01:51:53 WARNING mlflow.dspy.callback: Failed to create a span. Parent span not found.
2025/10/31 01:51:53 WARNING mlflow.dspy.callback: Failed to create a span. Parent span not found.
2025/10/31 01:51:57 WARNING mlflow.dspy.callback: Failed to end a span. Span not found for call_id: 4526c6c0ef7548e0a32546db418a48eb
2025/10/31 01:51:57 WARNING mlflow.dspy.callback: Failed to end a span. Span not found for call_id: 4526c6c0ef7548e0a32546db418a48eb
2025/10/31 01:52:01 WARNING dspy.utils.callback: Error when calling callback <mlflow.dspy.callback.MlflowCallback object at 0x000001F8670BE6D0>: 'ChainOfThought' object has no attribute 'signature'
2025/10/31 01:52:01 WARNING dspy.utils.callback: Error when calling callback <mlflow.dspy.callback.MlflowCallback object at 0x000001F8670BE6D0

Optimized - Average Score: 90.00%

🎉 Improvement: 0.0%


## 7. Inspect Optimized Prompts

In [13]:
# Test the optimized agent
test_question = "Explain the process of photosynthesis"
test_context = "Photosynthesis is the process by which green plants use sunlight to synthesize foods from carbon dioxide and water."

result = optimized_teacher(
    question=test_question,
    context=test_context,
    language="Tamil"
)

print("Question:", test_question)
print("\nResponse:")
print(result.response)

2025/10/31 01:53:21 WARNING dspy.utils.callback: Error when calling callback <mlflow.dspy.callback.MlflowCallback object at 0x000001F8670BE6D0>: 'ChainOfThought' object has no attribute 'signature'
2025/10/31 01:53:21 WARNING mlflow.dspy.callback: Failed to create a span. Parent span not found.
2025/10/31 01:53:21 WARNING mlflow.dspy.callback: Failed to create a span. Parent span not found.
2025/10/31 01:53:32 WARNING mlflow.dspy.callback: Failed to end a span. Span not found for call_id: a94f58e877cb4d96a0955d5e0ec2d906
2025/10/31 01:53:32 WARNING mlflow.dspy.callback: Failed to end a span. Span not found for call_id: a94f58e877cb4d96a0955d5e0ec2d906


Question: Explain the process of photosynthesis

Response:
Photosynthesis is the process where green plants use sunlight to make their food. They take in carbon dioxide from the air and water from the soil. Using sunlight, they convert these into glucose (a type of sugar) and oxygen. This process is vital for plants and produces oxygen for us to breathe.

பசுமை செடிகள் சூரிய ஒளியை பயன்படுத்தி தங்கள் உணவை உருவாக்கும் செயல்முறை. அவைகள் காற்றில் இருந்து கார்பன் டயாக்சைடு மற்றும் மண்ணில் இருந்து நீரை எடுத்துக்கொள்கின்றன. சூரிய ஒளியின் உதவியுடன், அவைகள் இதனை கிளுக்கோஸ் (ஒரு வகை சர்க்கரை) மற்றும் ஆக்சிஜனாக மாற்றுகின்றன. இந்த செயல்முறை செடிகளுக்கு மிகவும் முக்கியமானது மற்றும் நமக்கு மூச்சு எடுக்க ஆக்சிஜனை உருவாக்குகிறது.


## 8. Log to MLflow Model Registry

In [14]:
with mlflow.start_run(run_name="teacher_agent_v1"):
    # Log model
    model_info = mlflow.dspy.log_model(
        optimized_teacher,
        artifact_path="teacher_agent",
        input_example="What is mitosis?"
    )
    
    # Log metrics
    mlflow.log_metric("baseline_score", baseline_score)
    mlflow.log_metric("optimized_score", optimized_score)
    mlflow.log_metric("improvement_percent", improvement)
    
    # Log parameters
    mlflow.log_param("optimizer", "SIMBA")
    mlflow.log_param("max_demos", 3)
    mlflow.log_param("training_size", len(trainset))
    
    print("✓ Model logged to MLflow")
    print(f"  Model URI: {model_info.model_uri}")

2025/10/31 01:53:40 WARNING mlflow.models.signature: Failed to infer the model signature from the input example. Reason: TypeError("TeacherAgentDSPy.forward() missing 2 required positional arguments: 'question' and 'language'"). To see the full traceback, set the logging level to DEBUG via `logging.getLogger("mlflow").setLevel(logging.DEBUG)`. To disable automatic signature inference, set `signature` to `False` in your `log_model` or `save_model` call.
2025/10/31 01:53:53 WARNING mlflow.utils.requirements_utils: Failed to run predict on input_example, dependencies introduced in predict are not captured.
TypeError("TeacherAgentDSPy.forward() missing 2 required positional arguments: 'question' and 'language'")Traceback (most recent call last):


  File "c:\Users\benja\anaconda3\Lib\site-packages\mlflow\utils\_capture_modules.py", line 166, in load_model_and_predict
    model.predict(input_example, params=params)


  File "c:\Users\benja\anaconda3\Lib\site-packages\mlflow\dspy\wrapper.py"

2025/10/31 01:53:55 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": "What is mitosis?"
}. Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model uri and serving input example. A serving input example can be generated from model input example using `mlflow.models.convert_input_example_to_serving_input` function.
Got error: TeacherAgentDSPy.forward() missing 2 required positional arguments: 'question' and 'language'


Uploading artifacts:   0%|          | 0/7 [00:00<?, ?it/s]

✓ Model logged to MLflow
  Model URI: runs:/0b5f733dbbdf42339fff50833e273d46/teacher_agent
🏃 View run teacher_agent_v1 at: https://dbc-847d0b13-8d35.cloud.databricks.com/ml/experiments/3840367859585746/runs/0b5f733dbbdf42339fff50833e273d46
🧪 View experiment at: https://dbc-847d0b13-8d35.cloud.databricks.com/ml/experiments/3840367859585746
🏃 View run teacher_agent_v1 at: https://dbc-847d0b13-8d35.cloud.databricks.com/ml/experiments/3840367859585746/runs/0b5f733dbbdf42339fff50833e273d46
🧪 View experiment at: https://dbc-847d0b13-8d35.cloud.databricks.com/ml/experiments/3840367859585746


## 9. Register Model for Production

In [15]:
# Register model
model_name = "teacher-agent-dspy"

model_version = mlflow.register_model(
    model_uri=model_info.model_uri,
    name=model_name
)

print(f"✓ Registered model: {model_name}")
print(f"  Version: {model_version.version}")

RestException: INVALID_PARAMETER_VALUE: Bad model name: please specify all three levels of the model in the form `catalog_name.schema_name.model_name`

In [ ]:
# Transition to production
client = mlflow.tracking.MlflowClient()
client.transition_model_version_stage(
    name=model_name,
    version=model_version.version,
    stage="Production"
)

print(f"✓ Model transitioned to Production stage")

## 10. Load and Test Production Model

In [ ]:
# Load production model
production_model_uri = f"models:/{model_name}/production"
production_agent = mlflow.dspy.load_model(production_model_uri)

# Test it
test_result = production_agent(
    question="What is DNA replication?",
    context="DNA replication is the process of producing two identical replicas from one original DNA molecule.",
    language="Hindi"
)

print("Production Model Response:")
print(test_result.response)

## Next Steps

1. **Optimize other agents**: MCQ Solver, Mentor, Quiz Generator
2. **Expand training data**: Use more NEET materials
3. **Try other optimizers**: MIPROv2, BootstrapFewShot
4. **Fine-tune models**: Use BootstrapFinetune for Gemma
5. **Deploy to production**: Update app.py to load from MLflow

See `DSPy_Integration_Guide.md` for detailed instructions!